In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [67]:
file_name = "/Users/dafirebanks/Downloads/SongCSV.csv"
df = pd.read_csv(file_name)

In [5]:
df.columns

Index(['SongNumber', 'SongID', 'AlbumID', 'AlbumName', 'ArtistFamiliarity',
       'ArtistHotttnesss', 'ArtistID', 'ArtistLatitude', 'ArtistLocation',
       'ArtistLongitude', 'ArtistName', 'BarsConfidence', 'BarsStart',
       'BeatsConfidence', 'BeatsStart', 'Danceability', 'Duration',
       'EndOfFadeIn', 'Energy', 'KeySignature', 'KeySignatureConfidence',
       'Loudness', 'Mode', 'ModeConfidence', 'SectionsConfidence',
       'SectionsStart', 'SegmentsConfidence', 'SegmentsLoudnessMax',
       'SegmentsLoudnessMaxTime', 'SegmentsLoudnessMaxStart',
       'SegmentsPitches', 'SegmentsStart', 'SegmentsTimbre', 'SongHotttnesss',
       'TatumsConfidence', 'TatumsStart', 'Tempo', 'TimeSignature',
       'TimeSignatureConfidence', 'Title', 'Year'],
      dtype='object')

In [73]:
#### COLUMNS THAT ARE LISTS 
# Bars Confidence (99,)
# Bars Start (99,)

# Beats Confidence (397,)
# Beats Start (397,)

# Sections Confidence (10,)
# Sections Start (10,)

# SegmentsConfidence (935,)
# SegmentsLoudness max time (935,)
# SegmentsLoudness start (935, )
# Segments Start (935,)

# SegmentsPitches (935, 12)
# Segments Timbre (935, 12)

# Tatums confidence (794,)
# Tatums Start (794,)

In [68]:
df = df.drop(df["Energy"])

In [69]:
list_columns = ["BarsConfidence", "BarsStart", "BeatsConfidence", "BeatsStart", "SectionsConfidence", "SectionsStart", 
                "SegmentsConfidence", "SegmentsLoudnessMax", "SegmentsLoudnessMaxTime", "SegmentsLoudnessMaxStart", "SegmentsPitches", 
                "SegmentsStart", "SegmentsTimbre", "TatumsConfidence", "TatumsStart"]

def transform_str_to_list(df, columns):
    for col in columns:
        all_cells = []
        
        # Modify each cell 
        for i in range(1, len(df[col]) + 1):
            # Get list
            horrid_string = df[col][i]
            
            # Remove non-float values
            pretty_string = horrid_string.replace("[", "").replace("]", "").replace("...", "")
            
            # Convert to a list of floats
            pretty_list = pretty_string.split()
            try:
                beautiful_list = np.array([float(el) for el in pretty_list])
            except:
                print(col, i)
                print(pretty_list)
            
            # Add to list of cells
            all_cells.append(beautiful_list)
        
        # Make it a new column
        df[col] = pd.Series(all_cells)

In [70]:
transform_str_to_list(df, list_columns)

In [72]:
df["BarsStart"][2]

array([  0.39309,   2.75109,   5.13649,   7.52978,   9.95582,  12.38461,
        14.80287,  18.76487,  21.62017,  24.19393,  26.55564,  29.88044,
        32.18203,  34.56346,  37.12946,  39.54937,  41.94023,  44.30982,
        46.63997,  49.02193,  51.39808,  53.76505,  56.1252 ,  58.2811 ,
        60.47129,  62.76252,  65.14527,  67.53328,  69.66937,  71.78012,
        74.14577,  76.53773,  78.89286,  81.34618,  83.84617,  86.35194,
        88.71759,  91.05417,  93.45012,  95.64963,  98.01096, 100.34742,
       102.66176, 104.84234, 107.05525, 109.32195, 111.55548, 113.7171 ,
       115.97854, 118.25863, 120.53234, 122.83515, 125.12435, 127.39863,
       129.67619, 131.96568, 134.27184, 136.6043 , 138.94728, 141.27316,
       143.60168, 146.02489, 148.65472, 151.13881, 153.52388, 155.89711])

In [22]:
# Transform all arrays to the minimum
def reduce_arrays(col_name):
    col = df[col_name]
    
    min_len = len(col[1])
    
    # Get minimum length 
    for cell in col:
        if len(cell) < min_len:
            min_len = len(cell)
    
    print("minimum length is", min_len)
    
    for cell in col:
    
    # Transform all arrays in the column to the minimum length
    for i in range(1, len(col) + 1):
        col[i] = col[i][:min_len]
    
    return col

In [23]:
df["BarsConfidence"] = reduce_arrays("BarsConfidence")

minimum length is 2


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [32]:
da = df["BarsStart"][1]

In [ ]:
# Preprocessing
X = []
y = []

print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)